# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
from utils.ml import *
import winsound

# Notes
---
## 2022_09_02

### Observations

1. No need to make 10 transactions a day. Just one is enough

1. lema or sema match
    1. Using lema match works fine in both opening 
    1. Using lema has better results
    1. 144 produces better resuls over higher numbers like 1440 

1. Lema angle in order and close
    1. Using lema angle in order, delays order, but overall profits are high
    1. Using lema angle in close also increases overall profit

1. Before everything need to find why 2021 is consistently producing negative results in all the setups.
    1. What is that this causing this issue, how can it be addressed with in the same system
    1. less complex method is essential
    

### To Test
1. Find whether the opening or closing that is causing the most issue in loss years
1. Test other years to understand how well this setup works.
1. Remove lema match check from closing. This is delaying bad trasactions
1. Test 5 round up on lema match. Transactions are opening in lema match gaps
1. Ideation
    1. Test lema Bollinger bands to understand flow direction
    1. Test overall flow direction

---

## 2022_09_02

### Observations
1. Opening and closing need to happen faster. Cant close after ema match
1. Lema angle helps save positions against frequent flips but delays closing and causes losses
1. Market is non-directional (Dec 2021 is a good example) Goes up and down, when algo tries to open and close its always too late
1. Need a faster method


### To Test
1. Should test plotting close as a line instead of candles. Perform match in close with wider window
1. Close chart needs to be analysed. This looks promising.
1. Go with simple rolling min max for close. Works better than resample min max merge

---


## 2022_09_13

### Observations
1. Lema crossing lema max and close using sema crossing lema max (This is the highest yielding 2020)
1. Need to analyze the gap between lema and lema max as an indicator for profit and loss


---

## Result by Years

| Year | Angle open | Angle close | Lema match open | Lema match close | Result type | pips  |
| ---- | ---------- | ----------- | --------------- | ---------------- | ----------- | ----- |
| 2022 | Yes        | No          | Yes             | Yes              | Profit      | 577   |
| 2021 | Yes        | No          | Yes             | Yes              | Loss        | \-596 |
| 2020 | Yes        | No          | Yes             | Yes              | Profit      | 1462  |
| 2019 | Yes        | No          | Yes             | Yes              | Loss        | 193   |
| 2018 | Yes        | No          | Yes             | Yes              | Profit      | 60    |
| 2017 | Yes        | No          | Yes             | Yes              | Profit      | \-1396|    

data                = get_date_list(data)
data                = read_data(data)

In [2]:
data["input_year"] = 2021
data['start_date'] = {'year':data["input_year"], 'month':1, 'date':1}
data['end_date']   = {'year':data["input_year"], 'month':12, 'date':31}
data["product"] = 'eurusd'
data["input_file_name"] = f'{data["product"]}_{data["input_year"]}.csv'

In [3]:
%%time
data = run_engine(data)

pl   -0.05453
dtype: float64
--------------------------------------
            DateTime_frmt  order_side  close_type       pl
66646 2021-11-25 10:50:00         NaN  sema_close -0.00025
67228 2021-11-29 11:20:00         NaN  sema_close  0.00582
67428 2021-11-30 04:00:00         NaN  sema_close -0.00392
67732 2021-12-01 05:20:00         NaN  sema_close  0.00181
68514 2021-12-05 22:30:00         NaN  sema_close -0.00149
69179 2021-12-08 05:55:00         NaN  sema_close -0.00313
69572 2021-12-09 14:40:00         NaN  sema_close -0.00164
69948 2021-12-12 22:05:00         NaN  sema_close -0.00424
70847 2021-12-16 01:00:00         NaN  sema_close -0.00507
71332 2021-12-17 17:25:00         NaN  sema_close -0.00401
71615 2021-12-20 17:00:00         NaN  sema_close -0.00576
72562 2021-12-23 23:55:00         NaN  sema_close  0.00043
72794 2021-12-24 19:15:00         NaN  sema_close -0.00265
73342 2021-12-28 17:00:00         NaN  sema_close -0.00140
73651 2021-12-29 18:45:00         NaN  sema_clo

100%|██████████████████████████████████████| 234/234 [00:00<00:00, 1056.88it/s]

Wall time: 2min 23s


In [4]:
data['df_ohlc'].groupby(['close_type'])['pl'].sum()

close_type
sema_close   -0.05453
Name: pl, dtype: float64

In [5]:
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
print(x)
print('---------------------')
print(x.sort_values())
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')
monthly_avg = np.round(data['df_ohlc']['pl'].sum()/len(set(data['df_ohlc']['month_val'])),4)
print(f'monthly_avg : {monthly_avg}')

month_val
1    -0.01710
2     0.01056
3     0.01032
4    -0.01462
5     0.00671
6    -0.00275
7    -0.02189
8     0.02187
9    -0.00882
10   -0.01157
11    0.00551
12   -0.03275
Name: pl, dtype: float64
---------------------
month_val
12   -0.03275
7    -0.02189
1    -0.01710
4    -0.01462
10   -0.01157
9    -0.00882
6    -0.00275
11    0.00551
5     0.00671
3     0.01032
2     0.01056
8     0.02187
Name: pl, dtype: float64
---------------------
max : 8 : 0.02187
min : 12 : -0.03275
---------------------
monthly_avg : -0.0045


In [6]:
sys.exit()

SystemExit: 

In [ ]:
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl_type'].count().unstack())
print('----------------------------------------------')
print(data['df_small'].groupby(['weekday', 'pl_type'])['pl'].sum().unstack())

In [ ]:
data['lema_gap_pl_analysis_month'] = 3

data = lema_gap_pl_analysis(data)
data['lema_gap_pl_report']

In [ ]:
y = data['df_ohlc']['pl'].sum()
x = data['df_ohlc'].groupby(['month_val'])['pl'].sum()
send_telegram_message(f'{data["input_year"]}')
send_telegram_message(f'{y}')
send_telegram_message(f'{x}')
send_telegram_message(f'-------------------')

In [ ]:
y = data['df_ohlc'][data['df_ohlc']['month_val'] == 1]
x = y.groupby(['date_val'])['pl'].sum()
print(x)
print('---------------------')
print(f'max : {x.idxmax()} : {max(x)}')
print(f'min : {x.idxmin()} : {min(x)}')
print('---------------------')

In [ ]:
data['df_ohlc'].tail()

In [ ]:
data['df_ohlc']['lema_gap'].describe()

In [ ]:
x = 5
data['df_ohlc'].iloc[-5:,x:x+15]

In [ ]:
data['df_small']['lema_gap'].describe()

In [ ]:
a = np.arange(0,20)
for b in a:
    x = int(round(b/5.0)*5.0)
    print(b, x)

In [ ]:
abs(all['negative_sum']) / all['positive_sum']

In [ ]:
df_small['order_side']

In [ ]:
data['df_small'].to_csv('data/temp.csv')

In [24]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
chart_duration = 'month'
month = 12

# data['things_to_plot'] = ['tick', 'ema', 'BBands', 'dots']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'indicators']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'cross']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'tip']
# data['things_to_plot'] = ['ema', 'BBands', 'dots', 'positions', 'lema_angle']
# data['things_to_plot'] = ['ema', 'sema_angle', 'slema_angle', 'lema_angle', 'dir']
# data['things_to_plot'] = ['ema', 'positions', 'lema_angle','slema_angle','sema_angle', 'rsi']
# data['things_to_plot'] = ['ema','positions', 'lema_angle','lema_diff', 'lema_max_min']
# data['things_to_plot'] = ['candles','sema', 'slema','positions','lema', 'lema_angle', 'lema_max_min', 'close_diff']
# data['things_to_plot'] = ['lema', 'tip', 'candles', 'lema_angle', 'lema_change']
# data['things_to_plot'] = ['candles','sema', 'close','lema', 'lema_match', 'positions', 'lema_change']
# data['things_to_plot'] = ['close', 'sema','lema', 'lema_change', 'lema_match', 'close_max']
# data['things_to_plot'] = ['close', 'tip','lema', 'close_max', 'close_min', 'lema_match']
# data['things_to_plot'] = ['close', 'lema','lema_match', 'slema', 'positions']
# data['things_to_plot'] = ['close', 'sema', 'slema','lema','lema_match', 'tip']
# data['things_to_plot'] = ['close', 'sema','lema','lema_match', 'match_max', 'match_min', 'unmatched']
# data['things_to_plot'] = ['close', 'sema','lema','lema_match', 'lema_max_min', 'positions']
data['things_to_plot'] = ['close','lema', 'sema', 'lema_max_min', 'positions']

data['plot_start']  = f"{data['input_year']}-{month}-1"

if chart_duration == 'month':
    if month == 12:
        data['plot_stop']   = f"{data['input_year']+1}-1-1"
    else:
        data['plot_stop']   = f"{data['input_year']}-{month+1}-1"

elif chart_duration == 'year':        
    data['plot_start']  = f"{data['input_year']}-1-1"
    data['plot_stop']   = f"{data['input_year']+1}-1-1"


plot_graph(data)

generate_result_report(data)

In [ ]:
data['df_small'].columns

In [ ]:
temp

In [ ]:
temp['close']

In [ ]:
temp = data['df_small'][['pl_type', 'pl','order_side','close', 'lema_max', 'lema_min', 'lema', 'sema']].dropna()

temp.loc[temp['order_side'] == 'long', 'lema_diff'] = temp['lema'] - temp['lema_max']
temp.loc[temp['order_side'] == 'short', 'lema_diff'] = temp['lema_min'] - temp['lema']

temp.loc[temp['order_side'] == 'long', 'sema_diff'] = temp['sema'] - temp['lema_max']
temp.loc[temp['order_side'] == 'short', 'sema_diff'] = temp['lema_min'] - temp['sema']

temp.loc[temp['order_side'] == 'long', 'close_diff'] = temp['close'] - temp['lema_max']
temp.loc[temp['order_side'] == 'short', 'close_diff'] = temp['lema_min'] - temp['close']

temp.head()

In [ ]:
curve_type = 'sema_diff'

val_min = temp[curve_type].describe()['min'].round(5)
val_25 = temp[curve_type].describe()['25%'].round(5)
val_50 = temp[curve_type].describe()['50%'].round(5)
val_75 = temp[curve_type].describe()['75%'].round(5)
val_max = temp[curve_type].describe()['max'].round(5)

per_0_25 = temp.loc[(temp[curve_type] >= val_min) & (temp[curve_type] < val_25), 'pl'].sum().round(4)
per_25_50 = temp.loc[(temp[curve_type] >= val_25) & (temp[curve_type] < val_50), 'pl'].sum().round(4)
per_50_75 = temp.loc[(temp[curve_type] >= val_50) & (temp[curve_type] < val_75), 'pl'].sum().round(4)
per_75_100 = temp.loc[(temp[curve_type] >= val_75) & (temp[curve_type] <= val_max), 'pl'].sum().round(4)

per_0_25_count = temp.loc[(temp[curve_type] >= val_min) & (temp[curve_type] < val_25), 'pl'].count()
per_25_50_count = temp.loc[(temp[curve_type] >= val_25) & (temp[curve_type] < val_50), 'pl'].count()
per_50_75_count = temp.loc[(temp[curve_type] >= val_50) & (temp[curve_type] < val_75), 'pl'].count()
per_75_100_count = temp.loc[(temp[curve_type] >= val_75) & (temp[curve_type] <= val_max), 'pl'].count()

qrt = ['per_0_25' , 'per_25_50' , 'per_50_75' , 'per_75_100']
pl = [per_0_25 , per_25_50 , per_50_75 , per_75_100]
pl_count = [per_0_25_count , per_25_50_count , per_50_75_count , per_75_100_count]
qrt_start = [val_min, val_25, val_50, val_75]
qrt_stop = [val_25, val_50, val_75, val_max]

temp_report = pd.DataFrame(data = {'qrt':qrt, 'qrt_start':qrt_start, 'qrt_stop':qrt_stop, 'count':pl_count, 'pl':pl})
temp_report

In [ ]:
cols = ['DateTime_frmt', 'month_val', 'date_val', 'hour_val','open', 'high', 'low', 'close', 'candle_size', 'ask',
       'bid', 'lema', 'lema_angle', 'lema_angle_2', 'lema_angle_0', 'slema',
       'slema_angle', 'slema_angle_2', 'sema', 'sema_angle', 'sema_angle_2',
       'BBand_upper', 'BBand_middle', 'BBand_lower', 'BBand_width',
       'avg_BBand_width', 'adx', 'order_side', 'order_size', 'up', 'down', 'long_open', 'long_profit', 'long_loss', 'short_open',
       'short_profit', 'short_loss', 'close_type', 'pl', 'close_diff', 'tip','lema_match',  'lema_max',  'lema_min',  'lema_gap', 'lema_diff']

data['df_small'] = data['df_ohlc'][cols][data['df_ohlc'][['long_open', 'short_open', 'pl']].notnull().any(axis=1)].reset_index(drop=True)

data['df_small']['pl_type'] = np.nan
data['df_small'].iloc[0:50, 25:40]

In [8]:
temp = data['df_ohlc'].loc[~data['df_ohlc']['pl'].isna()]

In [21]:
x = temp.loc[temp['pl'] < 0,'pl'].round(4).value_counts()
l1 = x.index
l2 = x.values
df_neg = pd.DataFrame(data= {'pl':l1, 'count':l2})
df_neg['net_pl'] = df_neg['pl'] * df_neg['count']                   
df_neg = df_neg.sort_values(by = 'net_pl')                           
df_neg.head()

,pl,count,net_pl
7,-0.0062,2,-0.0124
8,-0.0058,2,-0.0116
0,-0.0028,4,-0.0112
12,-0.0052,2,-0.0104
15,-0.0048,2,-0.0096


In [23]:
x = temp.loc[temp['pl'] > 0,'pl'].round(4).value_counts()
l1 = x.index
l2 = x.values
df_pos = pd.DataFrame(data= {'pl':l1, 'count':l2})
df_pos['net_pl'] = df_neg['pl'] * df_neg['count']                   
df_pos = df_pos.sort_values(by = 'net_pl', ascending = False)                           
df_pos.head()

,pl,count,net_pl
0,0.0058,3,0.0174
25,0.0157,1,0.0157
24,0.0126,1,0.0126
20,0.0103,1,0.0103
17,0.0102,1,0.0102
